# 1. EDA

In [ ]:
import warnings
import numpy as np
import pandas as pd

from statsmodels.tsa.stattools import adfuller, acf, pacf, kpss
from statsmodels.stats.stattools import durbin_watson
from statsmodels.stats.diagnostic import acorr_ljungbox, het_arch
from statsmodels.tools.sm_exceptions import InterpolationWarning
import os
import math
import matplotlib.pyplot as plt


warnings.filterwarnings("ignore", category=InterpolationWarning)


def ts_diagnostics(
    df: pd.DataFrame,
    time_col: str = "date_id",
    max_lag: int = 60,
    season_search_max_lag: int = 60,
    season_min_acf: float = 0.30,
    alpha: float = 0.05,
    fill_gap: bool = True,
    save_path: str | None = None,
    # ▼ 추가 옵션
    contig_policy: str = "none",      # {"none","trim_ends","longest"}
    min_contig_len: int = 20,         # 유효구간 최소 길이
) -> pd.DataFrame:

    if time_col in df.columns:
        df = df.sort_values(time_col).reset_index(drop=True)

    value_cols = [c for c in df.columns if c != time_col]

    rows = []

    for col in value_cols:
        x_raw = df[col].astype(float).values
        n_tot = len(x_raw)

        # --- 연속 유효구간 선택 ---
        eff_start, eff_end = 0, n_tot - 1
        if contig_policy != "none":
            valid = np.isfinite(x_raw)

            if not valid.any():
                rows.append({"column": col, "length": n_tot, "effective_length": 0,
                             "coverage_ratio": 0.0, "used_range": None, "note": "all_nan"})
                continue

            if contig_policy == "trim_ends":
                # 앞/뒤 연속 NaN만 제거
                i0 = int(np.argmax(valid))                           # 첫 True
                i1 = int(n_tot - 1 - np.argmax(valid[::-1]))         # 마지막 True
            elif contig_policy == "longest":
                # 가장 긴 연속 True 구간
                arr = valid.astype(int)
                diff = np.diff(np.r_[0, arr, 0])
                starts = np.where(diff == 1)[0]
                ends   = np.where(diff == -1)[0]
                lengths = ends - starts
                k = int(np.argmax(lengths))
                i0, i1 = int(starts[k]), int(ends[k] - 1)
            else:
                i0, i1 = 0, n_tot - 1

            eff_start, eff_end = i0, i1
            if i1 - i0 + 1 < min_contig_len:
                rows.append({"column": col, "length": n_tot, "effective_length": i1 - i0 + 1,
                             "coverage_ratio": (i1 - i0 + 1) / max(n_tot, 1),
                             "used_range": f"{i0}:{i1}", "note": "contig_too_short"})
                continue

            x_work = x_raw[i0:i1+1]
        else:
            x_work = x_raw.copy()

        # --- 필요 시 보간/채움(선택된 구간 안에서만) ---
        if fill_gap:
            s = pd.Series(x_work)
            x = s.interpolate(method="linear", limit_direction="both").ffill().bfill().values
        else:
            x = x_work

        if np.sum(np.isfinite(x)) < 20:
            rows.append({"column": col, "length": n_tot, "effective_length": len(x),
                         "coverage_ratio": len(x)/max(n_tot,1), "used_range": f"{eff_start}:{eff_end}",
                         "note": "too_short"})
            continue

        # --- 통계 검정(기존 로직 그대로) ---
        def safe_adf(v):
            try:
                st, pv, *_ = adfuller(v, autolag="AIC")
                return st, pv
            except Exception:
                return np.nan, np.nan

        def safe_kpss_with_note(v):
            try:
                st, pv, _, _ = kpss(v, regression="c", nlags="auto")
                note = "p < 0.01 (beyond table range)" if pv <= 0.01 else ("p > 0.10 (beyond table range)" if pv >= 0.10 else None)
                return st, pv, note
            except Exception:
                return np.nan, np.nan, None

        def safe_acf(v):
            try:
                return acf(v, nlags=min(max_lag, len(v) - 2), fft=False)
            except Exception:
                return np.array([np.nan])

        def safe_pacf(v):
            try:
                return pacf(v, nlags=min(max_lag, len(v) - 2))
            except Exception:
                return np.array([np.nan])

        def safe_dw(v):
            try:
                return durbin_watson(v - np.mean(v))
            except Exception:
                return np.nan

        def safe_ljung_box(v):
            try:
                lags = min(max_lag, max(1, len(v) // 5))
                lb = acorr_ljungbox(v, lags=[lags], return_df=True)
                return float(lb["lb_stat"].values[0]), float(lb["lb_pvalue"].values[0])
            except Exception:
                return np.nan, np.nan

        def safe_arch_lm(v):
            try:
                lags = min(max_lag, int(np.sqrt(len(v))))
                st, pv, *_ = het_arch(v, nlags=lags)
                return st, pv
            except Exception:
                return np.nan, np.nan

        adf_stat, adf_p = safe_adf(x)
        kpss_stat, kpss_p, kpss_note = safe_kpss_with_note(x)
        acf_vals = safe_acf(x)
        pacf_vals = safe_pacf(x)
        dw = safe_dw(x)
        lb_stat, lb_p = safe_ljung_box(x)
        arch_stat, arch_p = safe_arch_lm(x)

        def classify_stationarity(adf_p, kpss_p):
            if np.isnan(adf_p) and np.isnan(kpss_p): return "unknown"
            adf_ok = (not np.isnan(adf_p)) and (adf_p < alpha)
            kpss_ok = (not np.isnan(kpss_p)) and (kpss_p >= alpha)
            if adf_ok and kpss_ok: return "stationary"
            if adf_ok or kpss_ok:  return "likely_stationary"
            return "non_stationary"

        def classify_autocorr(acf_vals, lb_p):
            acf1 = acf_vals[1] if isinstance(acf_vals, np.ndarray) and len(acf_vals) > 1 else np.nan
            if (not np.isnan(lb_p)) and (lb_p < alpha): return "autocorrelated"
            if (not np.isnan(acf1)) and (abs(acf1) >= 0.20): return "autocorrelated"
            return "no_autocorr"

        def classify_white_noise(stationarity_label, lb_p):
            if stationarity_label in ("stationary","likely_stationary") and (not np.isnan(lb_p)) and (lb_p >= alpha):
                return "white_noise"
            return "not_white_noise"

        def classify_heteroskedasticity(arch_p):
            if (not np.isnan(arch_p)) and (arch_p < alpha): return "heteroskedastic"
            if np.isnan(arch_p): return "unknown"
            return "homoskedastic"

        stationarity = classify_stationarity(adf_p, kpss_p)
        autocorr_lbl = classify_autocorr(acf_vals, lb_p)
        seasonal_flag = False; season_lag = np.nan; season_acf = np.nan
        if acf_vals is not None and len(acf_vals) > 2:
            seg = acf_vals[2:min(season_search_max_lag, len(acf_vals)-1)+1]
            if len(seg) > 0:
                k = np.nanargmax(seg)
                season_lag = int(2 + k); season_acf = float(seg[k])
                seasonal_flag = season_acf >= season_min_acf

        acf1 = float(acf_vals[1]) if isinstance(acf_vals, np.ndarray) and len(acf_vals) > 1 else np.nan
        pacf1 = float(pacf_vals[1]) if isinstance(pacf_vals, np.ndarray) and len(pacf_vals) > 1 else np.nan
        acf10 = float(acf_vals[min(10, len(acf_vals)-1)]) if isinstance(acf_vals, np.ndarray) and len(acf_vals) > 10 else np.nan

        rows.append({
            "column": col,
            "length": n_tot,
            "effective_length": len(x),
            "coverage_ratio": len(x) / max(n_tot, 1),
            "used_range": f"{eff_start}:{eff_end}",
            "ADF_stat": adf_stat, "ADF_p": adf_p,
            "KPSS_stat": kpss_stat, "KPSS_p": kpss_p, "KPSS_note": kpss_note,
            "stationarity_label": stationarity,
            "DW_stat": dw,
            "LB_stat": lb_stat, "LB_p": lb_p,
            "autocorr_label": autocorr_lbl,
            "ACF_lag1": acf1, "ACF_lag10": acf10, "PACF_lag1": pacf1,
            "seasonal_flag": bool(seasonal_flag),
            "season_period_est": season_lag,
            "season_acf_at_period": season_acf,
            "white_noise_label": classify_white_noise(stationarity, lb_p),
            "ARCH_stat": arch_stat, "ARCH_p": arch_p, "variance_label": classify_heteroskedasticity(arch_p),
        })

    results_df = pd.DataFrame(rows).set_index("column").sort_index()
    if save_path is not None:
        results_df.to_csv(save_path, index=True)
        print(f"Saved diagnostics to: {save_path}")
    return results_df



def ts_eda_overview(
    df: pd.DataFrame,
    time_col: str = "date_id",
    save_dir: str | None = "./eda_plots",
    max_hist_cols: int = 24,
    max_box_cols: int = 24,
    corr_method: str = "pearson",
    sample_rows_for_plots: int | None = None,
    nan_heatmap_row_sample: int | None = None,  # None이면 전체 행
    show: bool = False,
    label_on: bool = True,
    chunk_size: int = 50,
    # ▶ 추가: NaN 히트맵 높이 제어
    nan_heatmap_height_per_row: float = 0.02,
    nan_heatmap_base_height: float = 3.0,
    nan_heatmap_max_height: float = 20.0,   # ← 최대 높이(인치)
):
    """
    시계열/표형 데이터의 전체적 분포/결측/상관을 빠르게 시각화합니다.

    변경사항
    - 박스플롯: q95-q5 스케일을 로그 스케일 버킷(10의 지수)으로 묶어, 비슷한 단위끼리 표시
    - NaN 비율 바차트: 50개 단위로 분할 저장/표시
    - NaN 히트맵: 열 기준 50개 단위로 분할 저장/표시
    - 상관 히트맵: 열 기준 50개 단위로 분할 저장/표시

    Parameters
    ----------
    df : pd.DataFrame
    time_col : str
    save_dir : str|None
    max_hist_cols : int
    max_box_cols : int
    corr_method : {"pearson","spearman"}
    sample_rows_for_plots : int|None
    nan_heatmap_row_sample : int
    show : bool
    label_on : bool
    chunk_size : int
    """
    # ----------------------------
    # 준비
    # ----------------------------
    if save_dir is not None:
        os.makedirs(save_dir, exist_ok=True)

    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    if time_col in numeric_cols:
        numeric_cols.remove(time_col)

    plot_df = df.copy()
    if sample_rows_for_plots is not None and len(plot_df) > sample_rows_for_plots:
        plot_df = plot_df.sample(sample_rows_for_plots, random_state=42).sort_index()

    # 헬퍼: 컬럼 리스트를 고정 길이로 자르기
    def chunks(lst, n):
        for i in range(0, len(lst), n):
            yield lst[i:i+n]

    # ----------------------------
    # 1) 결측 비율 요약 테이블 + 바차트(청크)
    # ----------------------------
    na_ratio = df.isna().mean().rename("nan_ratio")
    desc = df.describe(include="all").T
    summary_df = desc.join(na_ratio, how="outer")
    summary_df = summary_df.sort_values("nan_ratio", ascending=False)
    if save_dir is not None:
        summary_df.to_csv(os.path.join(save_dir, "summary_numeric_and_nan.csv"))

    # NaN bar: 50개씩 끊어서
    ordered_cols = na_ratio.sort_values(ascending=False).index.tolist()
    for j, cols_chunk in enumerate(chunks(ordered_cols, chunk_size), start=1):
        plt.figure(figsize=(12, 5))
        na_ratio.loc[cols_chunk].plot(kind="bar")
        if label_on:
            plt.title(f"NaN ratio per column (chunk {j})")
            plt.ylabel("ratio")
            plt.xlabel("columns")
            plt.xticks(rotation=45, ha="right", fontsize=8)
        else:
            plt.title(""); plt.ylabel(""); plt.xlabel("")
            plt.xticks([]); plt.yticks([])
        plt.tight_layout()
        if save_dir is not None:
            plt.savefig(os.path.join(save_dir, f"nan_ratio_bar_chunk{j}.png"), dpi=150)
        if show: plt.show()
        plt.close()

    # ----------------------------
    # 2) 히스토그램 (상위 max_hist_cols개)
    # ----------------------------
    hist_cols = numeric_cols[:max_hist_cols]
    if len(hist_cols) > 0:
        n = len(hist_cols)
        ncols = min(6, max(1, int(math.ceil(n ** 0.5))))
        nrows = int(math.ceil(n / ncols))
        plt.figure(figsize=(3.2 * ncols, 2.6 * nrows))
        for i, c in enumerate(hist_cols, 1):
            ax = plt.subplot(nrows, ncols, i)
            plot_df[c].dropna().hist(bins=40, ax=ax)
            if label_on:
                ax.set_title(c, fontsize=9)
                ax.set_ylabel("freq")
            else:
                ax.set_title(""); ax.set_xticks([]); ax.set_yticks([])
        plt.tight_layout()
        if save_dir is not None:
            plt.savefig(os.path.join(save_dir, "histograms.png"), dpi=150)
        if show: plt.show()
        plt.close()

    # ----------------------------
    # 3) 박스플롯 (스케일 버킷으로 그룹)
    # ----------------------------
    # 스케일 = q95 - q5 (robust range). 0 또는 NaN이면 'const/zero' 버킷.
    scale_info = {}
    for c in numeric_cols:
        x = plot_df[c].astype(float).values
        x = x[~np.isnan(x)]
        if x.size == 0:
            rng = np.nan
        else:
            q5, q95 = np.nanpercentile(x, [5, 95])
            rng = float(q95 - q5)
        scale_info[c] = rng

    def bucket_of(rng):
        if rng is None or np.isnan(rng) or rng <= 0:
            return "const/zero"
        exp = int(np.floor(np.log10(rng)))
        # 예: 10^-3 ~ 10^-2 등
        return f"1e{exp}~1e{exp+1}"

    buckets = {}
    for c, rng in scale_info.items():
        b = bucket_of(rng)
        buckets.setdefault(b, []).append(c)

    # 버킷별로 정렬(상수/제로 먼저), 그 다음 지수 오름차순
    def bucket_key(b):
        if b == "const/zero": return -999
        # "1e-3~1e-2" 형태에서 -3 추출
        try:
            exp = int(b.split("~")[0].replace("1e", ""))
        except:
            exp = 0
        return exp

    for b in sorted(buckets.keys(), key=bucket_key):
        cols_in_bucket = buckets[b]
        if len(cols_in_bucket) == 0:
            continue
        # 페이지 단위로 나눠 그리기
        for j, cols_chunk in enumerate(chunks(cols_in_bucket, max_box_cols), start=1):
            plt.figure(figsize=(max(10, 0.4 * len(cols_chunk)), 5))
            plot_df[cols_chunk].boxplot(rot=45 if label_on else 0)
            if label_on:
                plt.title(f"Boxplots (bucket={b}, chunk={j})")
            else:
                plt.title(""); plt.xlabel(""); plt.ylabel("")
                plt.xticks([]); plt.yticks([])
            plt.tight_layout()
            if save_dir is not None:
                fn = f"boxplots_bucket_{b.replace('/', '_').replace('~','to').replace('-','m')}_chunk{j}.png"
                plt.savefig(os.path.join(save_dir, fn), dpi=150)
            if show: plt.show()
            plt.close()

    # ----------------------------
    # 4) 상관 히트맵 (열 50개씩 끊어서)
    # ----------------------------
    corr_cols_all = numeric_cols[:]
    if len(corr_cols_all) >= 2:
        for j, cols_chunk in enumerate(chunks(corr_cols_all, chunk_size), start=1):
            corr = plot_df[cols_chunk].corr(method=corr_method)
            plt.figure(figsize=(min(12, 0.25*len(cols_chunk)+3), min(12, 0.25*len(cols_chunk)+3)))
            im = plt.imshow(corr.values, aspect="auto")
            plt.colorbar(im, fraction=0.045, pad=0.04)
            if label_on:
                plt.xticks(range(len(cols_chunk)), cols_chunk, rotation=90, fontsize=7)
                plt.yticks(range(len(cols_chunk)), cols_chunk, fontsize=7)
                plt.title(f"Correlation heatmap ({corr_method}) - chunk {j}")
            else:
                plt.xticks([]); plt.yticks([]); plt.title("")
            plt.tight_layout()
            if save_dir is not None:
                plt.savefig(os.path.join(save_dir, f"correlation_heatmap_chunk{j}.png"), dpi=150)
            if show: plt.show()
            plt.close()

    # ----------------------------
    # 5) NaN 히트맵 (행 전체 or 샘플)
    # ----------------------------
    nan_mat = df.isna()
    if isinstance(nan_heatmap_row_sample, int) and nan_heatmap_row_sample > 0:
        if len(nan_mat) > nan_heatmap_row_sample:
            nan_mat = nan_mat.head(nan_heatmap_row_sample)
    
    all_cols = df.columns.tolist()
    
    for j, cols_chunk in enumerate(chunks(all_cols, chunk_size), start=1):
        sub = nan_mat[cols_chunk]
        n_rows, n_cols = sub.shape
    
        # 🔧 높이 계산 + 상한 적용 (여기만 사용)
        height = nan_heatmap_height_per_row * n_rows + nan_heatmap_base_height
        height = min(height, nan_heatmap_max_height)
    
        width = min(12, 0.25 * n_cols + 3)
    
        fig, ax = plt.subplots(figsize=(width, height))
        im = ax.imshow(sub.values, aspect="auto")
        if label_on:
            ax.set_title(f"NaN heatmap (rows={n_rows}) - chunk {j}")
            ax.set_xlabel("columns")
            ax.set_ylabel("rows")
            ax.set_xticks(range(n_cols))
            ax.set_xticklabels(cols_chunk, rotation=90, fontsize=7)
            ax.set_yticks([])  # 행 라벨은 생략
        else:
            ax.set_title(""); ax.set_xlabel(""); ax.set_ylabel("")
            ax.set_xticks([]); ax.set_yticks([])
    
        plt.tight_layout()
        if save_dir is not None:
            plt.savefig(os.path.join(save_dir, f"nan_heatmap_chunk{j}.png"), dpi=150)
        if show:
            plt.show()
        plt.close(fig)


    return summary_df

In [ ]:
# === Show ALL columns with your EDA helpers (no file saving, inline plots) ===
import pandas as pd
import numpy as np

# -------------------------------------------------
# 0) Load data
# -------------------------------------------------
PATH = "/kaggle/input/hull-tactical-market-prediction/train.csv"
time_col = "date_id"
df = pd.read_csv(PATH)

# -------------------------------------------------
# 1) Determine numeric columns (excluding time_col)
# -------------------------------------------------
num_cols = df.select_dtypes(include=np.number).columns.tolist()
if time_col in num_cols:
    num_cols.remove(time_col)
N = len(num_cols)
print(f"Numeric columns (excluding '{time_col}'): {N}")

# -------------------------------------------------
# 2) Run EDA plots inline (nothing saved)
#    - Uses your ts_eda_overview() already defined above
# -------------------------------------------------
_ = ts_eda_overview(
    df=df,
    time_col="date_id",
    save_dir=None,
    show=True,
    chunk_size=len(df.columns)  # ✅ 전체 한 번에
)

# -------------------------------------------------
# 3) Time-series diagnostics for ALL columns
#    - Uses your ts_diagnostics() already defined above
# -------------------------------------------------
diag_df = ts_diagnostics(
    df,
    time_col="date_id",
    fill_gap=False,
    contig_policy="longest",
    min_contig_len=180
)

# -------------------------------------------------
# 4) Summary table (describe + NaN ratio) for ALL columns
#    - ts_eda_overview returns this; re-run once with no plotting to get table
# -------------------------------------------------
summary_df = df.describe(include="all").T
summary_df["nan_ratio"] = df.isna().mean()

# -------------------------------------------------
# 5) Untruncate display to see ALL rows/columns
# -------------------------------------------------
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

print("\n=== Summary (describe + NaN ratio) ===")
display(summary_df)

print("\n=== Time-series diagnostics (all columns) ===")
display(diag_df)


In [ ]:
need_diff = diag_df.query("stationarity_label == 'non_stationary'").index.tolist()
has_season = diag_df.query("seasonal_flag == True").assign(period=lambda d: d['season_period_est'])[["period"]]
has_autocorr = diag_df.query("autocorr_label == 'autocorrelated' or LB_p < 0.05").index.tolist()
hetero = diag_df.query("variance_label == 'heteroskedastic'").index.tolist()
maybe_noise = diag_df.query("white_noise_label == 'white_noise'").index.tolist()

print("차분/변환 후보:", need_diff[:10], "…")
print("계절성 후보(+주기):\n", has_season.head())
print("자기상관 있음:", has_autocorr[:10], "…")
print("이분산 있음:", hetero[:10], "…")
print("화이트노이즈(신호 약함):", maybe_noise[:10], "…")

# 2. preprocessing

In [ ]:
# =========================================================
# Hull Tactical — Processing Utilities (functions only)
#  - NaN 50%↑ 컬럼 드롭 목록 계산
#  - 변환 대상 교집합 정리
#  - 차분/계절차분/래깅/롤링 z-score 적용 파이프라인
#  - 파일 I/O / 프린트 없음 (함수만 제공)
# =========================================================
from __future__ import annotations
from typing import List, Dict, Iterable, Tuple
import numpy as np
import pandas as pd


__all__ = [
    "rolling_zscore",
    "compute_nan_drop_columns",
    "resolve_transform_sets",
    "apply_pipeline",
]


# ---------- 0) 롤링 z-score (이분산 완화) ----------
def rolling_zscore(
    s: pd.Series,
    win: int = 63,
    min_frac: float = 0.34,
    eps: float = 1e-9,
) -> pd.Series:
    """
    롤링 평균/표준편차로 z-score 표준화.
    - min_periods = max(10, win * min_frac)
    """
    min_periods = max(10, int(round(win * float(min_frac))))
    m = s.rolling(win, min_periods=min_periods).mean()
    v = s.rolling(win, min_periods=min_periods).std()
    return (s - m) / (v + eps)


# ---------- 1) NaN 50%↑ 드롭 목록 계산 ----------
def compute_nan_drop_columns(
    train_df: pd.DataFrame,
    *,
    time_col: str = "date_id",
    nan_thresh: float = 0.50,
    train_only_targets: Iterable[str] = ("forward_returns", "risk_free_rate", "market_forward_excess_returns"),
    test_only_cols: Iterable[str] = ("is_scored", "lagged_forward_returns", "lagged_risk_free_rate", "lagged_market_forward_excess_returns"),
    extra_exempt: Iterable[str] | None = None,
) -> List[str]:
    """
    Train 기준 결측 비율 >= nan_thresh 인 컬럼 목록을 반환.
    - 시간/타깃/테스트 전용/extra_exempt 컬럼은 드롭 대상에서 제외.
    """
    exempt = set([time_col, *train_only_targets, *test_only_cols])
    if extra_exempt:
        exempt |= set(extra_exempt)

    nan_ratio = train_df.isna().mean()
    to_drop = [c for c, r in nan_ratio.items() if (r >= float(nan_thresh)) and (c not in exempt)]
    return to_drop


# ---------- 2) 변환 대상 교집합 정리 ----------
def resolve_transform_sets(
    df: pd.DataFrame,
    *,
    diff_cols_given: Iterable[str],
    seasonal_periods: Dict[str, int],
    autocorr_cols_given: Iterable[str],
    hetero_cols_given: Iterable[str],
) -> Tuple[List[str], Dict[str, int], List[str], List[str]]:
    """
    데이터프레임에 실제 존재하는 컬럼만 남겨 변환 계획을 정제.
    반환: (diff_cols, seasonal_info, lag_cols, hetero_cols)
    """
    cols = set(df.columns)

    def _intersect_list(cands: Iterable[str]) -> List[str]:
        return [c for c in cands if c in cols]

    diff_cols = _intersect_list(diff_cols_given)
    lag_cols = _intersect_list(autocorr_cols_given)
    hetero_cols = _intersect_list(hetero_cols_given)
    seasonal_info = {c: int(p) for c, p in seasonal_periods.items() if c in cols}

    return diff_cols, seasonal_info, lag_cols, hetero_cols


# ---------- 3) 전처리 파이프라인 ----------
def apply_pipeline(
    df: pd.DataFrame,
    *,
    time_col: str = "date_id",
    cols_to_drop: Iterable[str] = (),
    diff_cols: Iterable[str] = (),
    seasonal_info: Dict[str, int] = (),
    lag_cols: Iterable[str] = (),
    hetero_cols: Iterable[str] = (),
    white_noise_drop: Iterable[str] = (),
    z_win: int = 63,
    protect_train_only: Iterable[str] = ("forward_returns", "risk_free_rate", "market_forward_excess_returns"),
    protect_test_only: Iterable[str] = ("is_scored", "lagged_forward_returns", "lagged_risk_free_rate", "lagged_market_forward_excess_returns"),
    fill_eps: float = 0.0,
) -> pd.DataFrame:
    """
    차분/계절차분/래그/롤링 z-score를 적용하고 결측을 시계열 방식으로 처리.
    - drop: white_noise_drop ∪ cols_to_drop
    - diff: 지정 컬럼 치환(원본 대신 1차 차분)
    - seasonal: 지정 컬럼 치환(x_t - x_{t-p})
    - lag: _lag1 컬럼 추가(원본 유지)
    - hetero: _z{z_win} 컬럼 추가(원본 유지)
    - 결측: (피처) ffill → bfill → fillna(fill_eps), (보호 컬럼) bfill

    반환: 처리된 DataFrame (입력과 같은 컬럼 순서/정렬은 보장하지 않음)
    """
    out = df.sort_values(time_col).reset_index(drop=True).copy()

    # 0) 드롭
    drop_now = list({*list(cols_to_drop), *list(white_noise_drop)} & set(out.columns))
    if drop_now:
        out = out.drop(columns=drop_now, errors="ignore")

    # 1) 차분(치환)
    for c in diff_cols:
        if c in out.columns:
            out[c] = out[c].diff()

    # 2) 계절차분(치환)
    for c, p in seasonal_info.items():
        if c in out.columns:
            out[c] = out[c] - out[c].shift(int(p))

    # 3) 래깅 추가(원본 유지)
    for c in lag_cols:
        if c in out.columns:
            out[f"{c}_lag1"] = out[c].shift(1)

    # 4) 이분산 완화(z-score 추가, 원본 유지)
    for c in hetero_cols:
        if c in out.columns:
            out[f"{c}_z{int(z_win)}"] = rolling_zscore(out[c], win=int(z_win))

    # 5) 결측 처리
    protect_cols = {time_col, *protect_train_only, *protect_test_only}
    feat_cols = [c for c in out.columns if c not in protect_cols]

    if feat_cols:
        out[feat_cols] = out[feat_cols].ffill().bfill().fillna(fill_eps)

    for c in list(protect_cols & set(out.columns)):
        if out[c].dtype.kind in "fc":  # 수치형만
            out[c] = out[c].bfill()

    return out


# 3. train

In [ ]:
# ===== 여기는 위에서 제공된 평가식 그대로 사용 =====
MIN_INVESTMENT = 0
MAX_INVESTMENT = 2

class ParticipantVisibleError(Exception):
    pass

def score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str) -> float:
    solution = solution.copy()
    solution['position'] = submission['prediction']

    if solution['position'].max() > MAX_INVESTMENT:
        raise ParticipantVisibleError(f'Position of {solution["position"].max()} exceeds maximum of {MAX_INVESTMENT}')
    if solution['position'].min() < MIN_INVESTMENT:
        raise ParticipantVisibleError(f'Position of {solution["position"].min()} below minimum of {MIN_INVESTMENT}')

    solution['strategy_returns'] = solution['risk_free_rate'] * (1 - solution['position']) + solution['position'] * solution['forward_returns']

    strategy_excess_returns = solution['strategy_returns'] - solution['risk_free_rate']
    strategy_excess_cumulative = (1 + strategy_excess_returns).prod()
    strategy_mean_excess_return = (strategy_excess_cumulative) ** (1 / len(solution)) - 1
    strategy_std = solution['strategy_returns'].std()

    trading_days_per_yr = 252
    if strategy_std == 0:
        raise ZeroDivisionError
    sharpe = strategy_mean_excess_return / strategy_std * np.sqrt(trading_days_per_yr)
    strategy_volatility = float(strategy_std * np.sqrt(trading_days_per_yr) * 100)

    market_excess_returns = solution['forward_returns'] - solution['risk_free_rate']
    market_excess_cumulative = (1 + market_excess_returns).prod()
    market_mean_excess_return = (market_excess_cumulative) ** (1 / len(solution)) - 1
    market_std = solution['forward_returns'].std()

    market_volatility = float(market_std * np.sqrt(trading_days_per_yr) * 100)

    excess_vol = max(0, strategy_volatility / market_volatility - 1.2) if market_volatility > 0 else 0
    vol_penalty = 1 + excess_vol

    return_gap = max(0, (market_mean_excess_return - strategy_mean_excess_return) * 100 * trading_days_per_yr)
    return_penalty = 1 + (return_gap**2) / 100

    adjusted_sharpe = sharpe / (vol_penalty * return_penalty)
    return min(float(adjusted_sharpe), 1_000_000)

In [ ]:
# =========================================================
# Hull Tactical — LGBM Training Utilities (functions only)
#  - 파일 I/O / 제출 저장 / 데이터 로드는 포함하지 않음
#  - 학습에 필요한 함수만 제공합니다.
# =========================================================
# ===== 유틸 변경: risk_free_rate 반환 + CV에서 score() 사용 =====
from typing import Dict, List, Tuple, Iterable, Optional
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error

def default_lgbm_params(seed: int = 42) -> Dict:
    return dict(
        objective="regression",
        metric="rmse",
        learning_rate=0.019018772389315385,
        n_estimators=200,
        num_leaves=128,
        max_depth=6,
        min_child_samples=20,
        subsample=0.683544047260533,
        colsample_bytree=0.9617596652953478,
        reg_alpha=0.006879339704011885,
        reg_lambda=6.791029724043033,
        random_state=seed,
        verbose=-1,
        n_jobs=-1,
        # 필요하면: subsample_freq=1,
    )

def select_feature_columns(
    train_df: pd.DataFrame,
    test_df: pd.DataFrame,
    *,
    time_col: str = "date_id",
    train_only: Iterable[str] = ("forward_returns", "risk_free_rate", "market_forward_excess_returns"),
    test_only: Iterable[str] = ("is_scored","lagged_forward_returns","lagged_risk_free_rate","lagged_market_forward_excess_returns"),
) -> List[str]:
    ban = set([time_col, *train_only, *test_only])
    tr_feats = [c for c in train_df.columns if c not in ban]
    te_feats = [c for c in test_df.columns  if c not in ban]
    feats = sorted(list(set(tr_feats).intersection(te_feats)))
    return feats

def prepare_train_test_matrices(
    train_df: pd.DataFrame,
    test_df: pd.DataFrame,
    *,
    target_col: str = "forward_returns",
    time_col: str = "date_id",
    train_only: Iterable[str] = ("forward_returns", "risk_free_rate", "market_forward_excess_returns"),
    test_only:  Iterable[str] = ("is_scored","lagged_forward_returns","lagged_risk_free_rate","lagged_market_forward_excess_returns"),
) -> Tuple[pd.DataFrame, pd.Series, pd.DataFrame, List[str], pd.Series]:
    tr = train_df.sort_values(time_col).reset_index(drop=True)
    te = test_df.sort_values(time_col).reset_index(drop=True)

    feat_cols = select_feature_columns(tr, te, time_col=time_col, train_only=train_only, test_only=test_only)

    X = tr[feat_cols]
    y = tr[target_col].astype(float)
    X_test = te[feat_cols]
    # ⬇️ 추가: 검증 때 score() 계산용 risk-free
    rf = tr["risk_free_rate"].astype(float).reset_index(drop=True)

    return X, y, X_test, feat_cols, rf

def _pred_to_position(pred: np.ndarray) -> np.ndarray:
    """모델 예측을 [0,2] 포지션으로 매핑 (기본: 단순 clip)."""
    return np.clip(pred, MIN_INVESTMENT, MAX_INVESTMENT)

def time_series_cv_fit(
    X, y, X_test=None, *,
    risk_free_series=None,
    params=None,
    n_splits=5,
    early_stopping_rounds=500,   # ← 여기를 0이나 None으로 주면 ES 비활성화
    log_every_n=200,
    return_models=True,
    seed=42,
) -> Dict:
    if params is None:
        params = default_lgbm_params(seed=seed)

    tscv = TimeSeriesSplit(n_splits=n_splits)
    oof = np.zeros(len(X), dtype=float)
    preds_test = np.zeros(len(X_test), dtype=float) if X_test is not None else None

    models, imps, adj_scores = [], [], []

    for fold, (tr_idx, va_idx) in enumerate(tscv.split(X, y), 1):
        X_tr, y_tr = X.iloc[tr_idx], y.iloc[tr_idx]
        X_va, y_va = X.iloc[va_idx], y.iloc[va_idx]
        rf_va = None if risk_free_series is None else risk_free_series.iloc[va_idx].values

        model = lgb.LGBMRegressor(**params)

        # ✅ callbacks를 조건부로 구성
        cbs = [lgb.log_evaluation(log_every_n)]
        use_es = bool(early_stopping_rounds) and early_stopping_rounds > 0
        if use_es:
            cbs.insert(0, lgb.early_stopping(stopping_rounds=early_stopping_rounds, verbose=False))

        model.fit(
            X_tr, y_tr,
            eval_set=[(X_va, y_va)],
            eval_metric="rmse",
            callbacks=cbs,
        )

        # ✅ ES가 꺼져 있으면 전체 트리로 예측(num_iteration=None)
        num_it = model.best_iteration_ if use_es else None

        pred_va = model.predict(X_va, num_iteration=num_it)
        oof[va_idx] = pred_va

        if rf_va is not None:
            sol = pd.DataFrame({"forward_returns": y_va.values, "risk_free_rate": rf_va})
            sub = pd.DataFrame({"prediction": _pred_to_position(pred_va)})
            try:
                adj = score(sol, sub, row_id_column_name="row_id")
            except Exception:
                adj = -np.inf
            adj_scores.append(adj)
            print(f"[Fold {fold}] Adjusted Sharpe (metric): {adj:.6f}")

        if X_test is not None:
            preds_test += model.predict(X_test, num_iteration=num_it) / n_splits

        if return_models:
            models.append(model)
            try:
                imps.append(pd.Series(model.feature_importances_, index=X.columns, name=f"fold{fold}"))
            except Exception:
                pass

    rmse = float(mean_squared_error(y, oof, squared=False))
    out = {"oof": oof, "rmse": rmse, "preds_test": preds_test}
    if adj_scores:
        out["adj_sharpe_per_fold"] = adj_scores
        out["adj_sharpe_mean"] = float(np.mean(adj_scores))
    if return_models:
        out["models"] = models
        if imps:
            imp_df = pd.concat(imps, axis=1); imp_df["mean"] = imp_df.mean(axis=1)
            out["importances"] = imp_df
    return out

def fit_full_and_predict(
    X: pd.DataFrame,
    y: pd.Series,
    X_test: pd.DataFrame,
    *,
    params: Optional[Dict] = None,
    num_iteration: Optional[int] = None,
    seed: int = 42,
) -> Tuple[lgb.LGBMRegressor, np.ndarray]:
    if params is None:
        params = default_lgbm_params(seed=seed)
    model = lgb.LGBMRegressor(**params)
    model.fit(X, y)
    preds = model.predict(X_test, num_iteration=num_iteration)
    return model, preds

def feature_importance_df(models: List[lgb.LGBMRegressor], feature_names: Iterable[str]) -> pd.DataFrame:
    rows = []
    for i, m in enumerate(models, 1):
        try:
            rows.append(pd.Series(m.feature_importances_, index=list(feature_names), name=f"fold{i}"))
        except Exception:
            continue
    if not rows:
        return pd.DataFrame(index=list(feature_names))
    imp = pd.concat(rows, axis=1)
    imp["mean"] = imp.mean(axis=1)
    return imp.sort_values("mean", ascending=False)

In [ ]:
# =========================================
# Optuna tuner for LightGBM (Adjusted Sharpe)
# =========================================
import json, os
import numpy as np
import pandas as pd

# Optuna import (없으면 설치)
try:
    import optuna
except Exception:
    import sys, subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "optuna"])
    import optuna

# 안전: 포지션 클립 상수 (없으면 NameError)
MIN_INVESTMENT = 0.0
MAX_INVESTMENT = 2.0

# 대회 점수 함수가 없으면 RMSE로 폴백
try:
    _HAS_SCORE = callable(score)
except Exception:
    _HAS_SCORE = False

def _objective_factory(X, y, rf, n_splits=5, seed=42, early_stopping_rounds=500, log_every_n=200):
    def objective(trial: optuna.Trial) -> float:
        # ---- search space ----
        params = dict(
            objective="regression",
            metric="rmse",
            # 학습률/트리규모
            learning_rate=trial.suggest_float("learning_rate", 1e-4, 0.05, log=True),
            n_estimators=trial.suggest_int("n_estimators", 5_000, 40_000, step=1_000),
            num_leaves=trial.suggest_int("num_leaves", 31, 511, step=8),
            max_depth=trial.suggest_int("max_depth", 3, 12),
            min_child_samples=trial.suggest_int("min_child_samples", 5, 200),
            # 규제/샘플링
            subsample=trial.suggest_float("subsample", 0.4, 1.0),
            colsample_bytree=trial.suggest_float("colsample_bytree", 0.4, 1.0),
            reg_alpha=trial.suggest_float("reg_alpha", 1e-3, 100.0, log=True),
            reg_lambda=trial.suggest_float("reg_lambda", 1e-3, 100.0, log=True),
            random_state=seed,
            verbose=-1,
            n_jobs=-1,
        )

        # CV 학습 (네가 만든 함수 사용; 메트릭은 내부에서 계산)
        out = time_series_cv_fit(
            X, y, X_test=None,
            risk_free_series=rf,
            params=params,
            n_splits=n_splits,
            early_stopping_rounds=early_stopping_rounds,
            log_every_n=log_every_n,
            return_models=False,
            seed=seed,
        )

        # Optuna의 목표: 대회 메트릭 최대화 (없으면 -RMSE 최소화와 동치)
        if _HAS_SCORE and ("adj_sharpe_mean" in out):
            return float(out["adj_sharpe_mean"])   # ↑ maximize
        else:
            return -float(out["rmse"])             # ↑ maximize(음수 RMSE)

    return objective


def run_optuna_tuning(
    train_df: pd.DataFrame,
    test_df: pd.DataFrame,
    *,
    target_col="forward_returns",
    time_col="date_id",
    n_trials: int = 50,
    n_splits: int = 5,
    seed: int = 42,
    study_name: str = "lgbm_adj_sharpe",
    save_path: str = "/kaggle/working/best_lgbm_params.json",
):
    # 학습/추론 행렬 준비 (+ risk_free series)
    X, y, X_test, feat_cols, rf = prepare_train_test_matrices(
        train_df, test_df,
        target_col=target_col,
        time_col=time_col
    )

    # 목표 함수
    objective = _objective_factory(X, y, rf, n_splits=n_splits, seed=seed)

    # Pruner/샘플러(재현성 포함)
    sampler = optuna.samplers.TPESampler(seed=seed)
    pruner  = optuna.pruners.MedianPruner(n_warmup_steps=10)

    study = optuna.create_study(direction="maximize", study_name=study_name, sampler=sampler, pruner=pruner)
    study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

    print(f"[Optuna] Best value={study.best_value:.6f}")
    print("[Optuna] Best params:", study.best_params)

    # 저장 (서버가 읽을 수 있게)
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    with open(save_path, "w") as f:
        json.dump(study.best_params, f)
    print(f"[Optuna] Saved best params → {save_path}")

    return study


In [ ]:
"""
study = run_optuna_tuning(
    train_df, test_df,
    n_trials=60,       # 예: 60 트라이얼
    n_splits=5,        # TSCV 폴드 수
    seed=42,
    save_path="/kaggle/working/best_lgbm_params.json"
)
"""

# 4. submission

In [ ]:
# ----- Kelly & Volatility guard (add this near the other globals) -----
from collections import deque

# Kelly config
KELLY_WINDOW = 20       # 최근 영업일 정답률
KELLY_SHRINK = 1      # 축소계수: 멀티플 범위 [0.5, 1.5]

# Kelly state
_kelly_hits = deque(maxlen=KELLY_WINDOW)   # 1=정답, 0=오답
_prev_raw_pred_sign = None                 # 직전 예측 부호 (-1/0/+1)

def _kelly_multiplier_from_hits() -> float:
    """최근 정답률 p로 멀티플 계산: mult = 1 + KELLY_SHRINK*(2p-1) ∈ [1-KS, 1+KS]."""
    n = len(_kelly_hits)
    if n == 0:
        return 1.0
    p = (sum(_kelly_hits) + 1) / (n + 2)  # 라플라스 스무딩
    f = 2.0 * p - 1.0
    mult = 1.0 + KELLY_SHRINK * f
    return float(np.clip(mult, 1.0 - KELLY_SHRINK, 1.0 + KELLY_SHRINK))

# ---- Simple volatility guard (safe defaults) ----
_VOL_EWMA = None       # EWMA of strategy daily variance
_VOL_ALPHA = 0.1       # smoothing for variance
_VOL_SOFT_CAP = 0.24   # 연환산 24% 이상이면 감쇠 시작 (원하면 조정)

def _update_vol_guard(latest_df: pd.DataFrame, alloc: float) -> None:
    """어제 결과와 현재 배분으로 전략 일변동(제곱수익) EWMA 업데이트."""
    global _VOL_EWMA
    col = None
    if "lagged_forward_returns" in latest_df.columns:
        col = "lagged_forward_returns"
    elif "lagged_market_forward_excess_returns" in latest_df.columns:
        col = "lagged_market_forward_excess_returns"
    if col is None:
        return
    r = latest_df[col].iloc[0]
    if pd.isna(r):
        return
    strat_ret = float(alloc) * float(r)
    var = strat_ret * strat_ret
    _VOL_EWMA = var if _VOL_EWMA is None else (1 - _VOL_ALPHA) * _VOL_EWMA + _VOL_ALPHA * var

def _vol_damper() -> float:
    """연환산 변동성이 소프트 캡을 넘으면 1.0→0.6으로 선형 감쇠."""
    if _VOL_EWMA is None:
        return 1.0
    ann_vol = np.sqrt(_VOL_EWMA) * np.sqrt(252.0)
    if ann_vol <= _VOL_SOFT_CAP:
        return 1.0
    # 캡 대비 최대 2배에서 0.6까지 감쇠
    excess = np.clip((ann_vol - _VOL_SOFT_CAP) / _VOL_SOFT_CAP, 0.0, 1.0)
    return float(1.0 - 0.4 * excess)

In [ ]:
# =========================================================
# Hull Tactical — Online Inference Server
#  - LightGBM (Optuna best params) + 간소 파이프라인 유틸(폴백)
#  - A) Base 피처 세트
#  - B) Core(+exp) 파생 피처 Top-K (선택적, 기본은 A 사용)
#  - OoF Calibration: P(up) (Logit), μ (Linear)
#  - Kelly 레버리지 최대 2x + Vol-guard + 스무딩
# =========================================================
import os
from pathlib import Path
import time
from itertools import combinations
import numpy as np
import pandas as pd
import lightgbm as lgb
import kaggle_evaluation.default_inference_server
from sklearn.metrics import r2_score
from sklearn.linear_model import LogisticRegression, LinearRegression

# ---------------- Optional: Polars ----------------
try:
    import polars as pl
    _POLARS_AVAILABLE = True
except Exception as e:
    pl = None
    _POLARS_AVAILABLE = False
    print(f"[WARN] Polars unavailable; falling back to Pandas-only mode: {e}")

# =================================================
# 경로 & 상수
# =================================================
DATA_PATH = Path("/kaggle/input/hull-tactical-market-prediction")
TRAIN_CSV = DATA_PATH / "train.csv"
TEST_CSV  = DATA_PATH / "test.csv"  # local gateway 용
TIME_COL  = "date_id"
TARGET    = "forward_returns"

TRAIN_ONLY  = ("forward_returns","risk_free_rate","market_forward_excess_returns")
TEST_ONLY   = ("is_scored","lagged_forward_returns","lagged_risk_free_rate","lagged_market_forward_excess_returns")

# ==== Transform plan (same as diagnostics) ====
DIFF_COLS_GIVEN     = ['E10','E11','E12','E2','E20','E3','E5','E6','E9','I4']
SEASONAL_PERIODS    = {'D4':2,'D5':42,'D6':42,'D7':44,'D8':42}
AUTOCORR_COLS_GIVEN = ['D1','D2','D3','D4','D5','D6','D7','D8','D9','E1']
HETERO_COLS_GIVEN   = ['D1','D2','D3','D4','D5','D6','D7','D8','D9','E1']
WHITE_NOISE_DROP    = ['P3']  # 완전백색잡음으로 간주하여 제거할 후보

# NaN-drop threshold & z-score window
NAN_THRESH = 0.50
Z_WIN      = 63
FILL_EPS   = 0.0

# CV & model params
SEED         = 42
EARLY_STOP   = 300
LOG_EVERY    = 200
VAL_HORIZON  = 180     # 최근 180영업일 검증

# Allocation post-processing
CLIP_MIN = 0.0
CLIP_MAX = 2.0              # Kelly 포함 최종 레버리지 상한
SCALE_LOW_VOL  = 700.0
SCALE_HIGH_VOL = 500.0
SMOOTH_ALPHA   = 0.25
VOL_COL        = "V1"

# Vol guard (실현변동성 타겟)
TARGET_DAILY_VOL = float(os.getenv("TARGET_DAILY_VOL", "0.012"))  # 1.2%/day
VOL_GUARD_WIN    = int(os.getenv("VOL_GUARD_WIN", "90"))

# ====== Global state (online) ======
_models: list[lgb.LGBMRegressor] = []                  # A) base tuned models
_feat_cols: list[str] = []
_train_like_schema: list[str] = []
_drop_cols: list[str] = []
_diff_cols: list[str] = []
_seasonal_info: dict[str,int] = {}
_lag_cols: list[tuple[str,int]] = []     # (col, lag)
_hetero_cols: list[str] = []
_last_allocation: float = 0.0
_v1_median: float = 0.0
_const_cols: list[str] = []
_prev_raw_pred_sign = None
_kelly_hits = []
_test_buffer: pd.DataFrame = pd.DataFrame()

# For B) core+exp pruned tuned models (선택 사용 가능)
models_coreexp_sel: list[lgb.LGBMRegressor] = []
_feat_cols_coreexp_sel: list[str] = []

# ---- Optuna Best Params (provided) ----
_LGBM_TUNED = {
    "boosting_type": "gbdt",
    "max_depth": 6,
    "num_leaves": 97,
    "learning_rate": 0.04765455492382819,
    "min_child_samples": 606,
    "feature_fraction": 0.9267638600380922,
    "bagging_fraction": 0.5538379982286759,
    "bagging_freq": 7,
    "lambda_l1": 0.025572847585528788,
    "lambda_l2": 0.008280168455497349,
    "min_gain_to_split": 1.2392218344254143e-07,
}

# =================================================
# 유틸(폴백) — 프로젝트 내 전용 유틸이 있으면 import 하세요.
# =================================================
def default_lgbm_params(seed: int = 42) -> dict:
    """없으면 기본값."""
    return dict(
        objective="regression",
        metric="rmse",
        max_depth=-1,
        num_leaves=63,
        learning_rate=0.05,
        min_child_samples=20,
        feature_fraction=0.9,
        bagging_fraction=0.9,
        bagging_freq=1,
        lambda_l1=0.0,
        lambda_l2=0.0,
        min_gain_to_split=0.0,
    )

def compute_nan_drop_columns(
    df: pd.DataFrame,
    time_col: str,
    nan_thresh: float = 0.5,
    train_only_targets=(),
    test_only_cols=(),
    extra_exempt=None,
):
    exempt = set([time_col, *train_only_targets, *test_only_cols])
    if extra_exempt:
        exempt|= set(extra_exempt)
    drop = []
    for c in df.columns:
        if c in exempt: 
            continue
        frac = df[c].isna().mean()
        if frac >= nan_thresh:
            drop.append(c)
    # white-noise 후보 제거
    for c in WHITE_NOISE_DROP:
        if c in df.columns and c not in exempt and c not in drop:
            drop.append(c)
    return drop

def resolve_transform_sets(
    df: pd.DataFrame,
    diff_cols_given=None,
    seasonal_periods=None,
    autocorr_cols_given=None,
    hetero_cols_given=None,
):
    diff_cols = [c for c in (diff_cols_given or []) if c in df.columns]
    seasonal_info = {c:p for c,p in (seasonal_periods or {}).items() if c in df.columns}
    # 간단히 1,5,21 래그 생성
    lag_cols = []
    for c in (autocorr_cols_given or []):
        if c in df.columns:
            for L in (1,5,21):
                lag_cols.append((c, L))
    hetero_cols = [c for c in (hetero_cols_given or []) if c in df.columns]
    return diff_cols, seasonal_info, lag_cols, hetero_cols

def _zscore(x: pd.Series, win: int) -> pd.Series:
    r = x.rolling(win, min_periods=max(5, win//5))
    m = r.mean(); s = r.std(ddof=0)
    z = (x - m) / (s.replace(0.0, np.nan))
    return z

def apply_pipeline(
    df_in: pd.DataFrame,
    time_col: str,
    cols_to_drop=None,
    diff_cols=None,
    seasonal_info=None,
    lag_cols=None,
    hetero_cols=None,
    white_noise_drop=None,
    z_win=63,
    protect_train_only=(),
    protect_test_only=(),
    fill_eps=0.0,
):
    df = df_in.sort_values(time_col).reset_index(drop=True).copy()
    cols_to_drop = set(cols_to_drop or [])
    keep_all = set([time_col, *protect_train_only, *protect_test_only])

    # 1) Drop NaN-heavy & white noise
    extra_drop = set(white_noise_drop or [])
    drop_now = [c for c in df.columns if (c in cols_to_drop or c in extra_drop) and c not in keep_all]
    if drop_now:
        df = df.drop(columns=drop_now, errors="ignore")

    # 2) Diffs
    for c in (diff_cols or []):
        if c in df.columns:
            df[f"{c}_diff1"] = df[c].diff()

    # 3) Seasonality (rolling mean)
    for c, p in (seasonal_info or {}).items():
        if c in df.columns:
            df[f"{c}_ma{p}"] = pd.to_numeric(df[c], errors="coerce").rolling(p, min_periods=max(2, p//2)).mean()

    # 4) Lags
    for c, L in (lag_cols or []):
        if c in df.columns:
            df[f"{c}_lag{L}"] = df[c].shift(L)

    # 5) Heteroskedastic proxy (rolling std)
    for c in (hetero_cols or []):
        if c in df.columns:
            df[f"{c}_std{z_win}"] = pd.to_numeric(df[c], errors="coerce").rolling(z_win, min_periods=max(5, z_win//5)).std(ddof=0)

    # 6) Rolling z-score (보호 컬럼 제외)
    num_cols = [c for c in df.columns if c not in keep_all]
    for c in num_cols:
        if c == time_col: 
            continue
        s = pd.to_numeric(df[c], errors="coerce")
        df[c] = _zscore(s, z_win)

    # 마무리: 결측/무한치 채우기
    df = df.replace([np.inf, -np.inf], np.nan).fillna(0.0)
    return df

# =================================================
# 모델 파라미터 빌더
# =================================================
def build_lgbm_params(seed: int) -> dict:
    base = {}
    try:
        base = default_lgbm_params(seed=seed)
    except Exception:
        base = {}
    base.update(_LGBM_TUNED)
    # 모델 생성자에서 직접 넣을 값 제거 (중복/alias 방지)
    for k in ["random_state", "n_estimators", "n_jobs", "verbosity", "subsample", "colsample_bytree"]:
        base.pop(k, None)

    # (선택) num_leaves ≤ 2^max_depth - 1 강제
    ENFORCE_LEAF_DEPTH = False
    if ENFORCE_LEAF_DEPTH:
        md = base.get("max_depth", -1)
        if isinstance(md, int) and md > 0 and "num_leaves" in base:
            base["num_leaves"] = min(base["num_leaves"], (1 << md) - 1)
    return base

# =================================================
# 헬퍼
# =================================================
def _to_pandas(df_any) -> pd.DataFrame:
    if isinstance(df_any, pd.DataFrame):
        return df_any.copy()
    if _POLARS_AVAILABLE and isinstance(df_any, pl.DataFrame):
        return df_any.to_pandas()
    if isinstance(df_any, dict):
        return pd.DataFrame([df_any])
    try:
        return pd.DataFrame(df_any)
    except Exception:
        return pd.DataFrame([df_any])

def _online_pipeline_apply(df_in: pd.DataFrame) -> pd.DataFrame:
    out = apply_pipeline(
        df_in,
        time_col=TIME_COL,
        cols_to_drop=_drop_cols,
        diff_cols=_diff_cols,
        seasonal_info=_seasonal_info,
        lag_cols=_lag_cols,
        hetero_cols=_hetero_cols,
        white_noise_drop=WHITE_NOISE_DROP,
        z_win=Z_WIN,
        protect_train_only=TRAIN_ONLY,
        protect_test_only=TEST_ONLY,
        fill_eps=FILL_EPS,
    )
    if _const_cols:
        out = out.drop(columns=[c for c in _const_cols if c in out.columns], errors="ignore")
    return out

def _align_features(df_proc: pd.DataFrame) -> pd.DataFrame:
    for c in _feat_cols:
        if c not in df_proc.columns:
            df_proc[c] = 0.0
    return df_proc.reindex(columns=_feat_cols, fill_value=0.0)

def _choose_scale(row: pd.Series) -> float:
    try:
        v1 = float(row.get(VOL_COL, np.nan))
    except Exception:
        v1 = np.nan
    if np.isnan(v1):
        return SCALE_HIGH_VOL
    return SCALE_LOW_VOL if v1 < _v1_median else SCALE_HIGH_VOL

# =================================================
# Start-up: Train once
# =================================================
t0 = time.time()
train_raw = pd.read_csv(TRAIN_CSV).sort_values(TIME_COL).reset_index(drop=True)

# (1) Drop columns by NaN>=50% on TRAIN
_fast_drop = compute_nan_drop_columns(
    train_raw,
    time_col=TIME_COL,
    nan_thresh=NAN_THRESH,
    train_only_targets=TRAIN_ONLY,
    test_only_cols=TEST_ONLY,
    extra_exempt=None,
)

# (2) Resolve transform sets from TRAIN schema
_diff_cols, _seasonal_info, _lag_cols, _hetero_cols = resolve_transform_sets(
    train_raw,
    diff_cols_given=DIFF_COLS_GIVEN,
    seasonal_periods=SEASONAL_PERIODS,
    autocorr_cols_given=AUTOCORR_COLS_GIVEN,
    hetero_cols_given=HETERO_COLS_GIVEN,
)

_drop_cols = _fast_drop

# (3) Apply pipeline to TRAIN
train_proc = _online_pipeline_apply(train_raw)

# (4) Choose base features
ban = set([TIME_COL, *TRAIN_ONLY, *TEST_ONLY])
_feat_cols = sorted([c for c in train_proc.columns if c not in ban])
_train_like_schema = train_proc.columns.tolist()

# Compute V1 median for scaling (if available)
_v1_median = float(pd.to_numeric(train_proc[VOL_COL], errors="coerce").median()) if (VOL_COL in train_proc.columns) else 0.0

# (5) Build folds: 5 expanding folds, 180-day validation windows
X_base = train_proc[_feat_cols]
y      = train_proc[TARGET].astype(float)

# remove constant features in Base
_const_cols = [c for c in _feat_cols if X_base[c].nunique(dropna=False) <= 1]
if _const_cols:
    X_base = X_base.drop(columns=_const_cols)
    _feat_cols = [c for c in _feat_cols if c not in _const_cols]

K = 5
H = VAL_HORIZON
N = len(X_base)
folds = []
for k in range(K):
    val_start = max(0, N - H * (K - k))
    val_end   = min(N, val_start + H)
    train_end = val_start
    if train_end > 0 and (val_end - val_start) > 0:
        folds.append((np.arange(0, train_end), np.arange(val_start, val_end)))
if not folds:
    val_size = min(H, max(30, int(N * 0.15)))
    folds = [(np.arange(0, N - val_size), np.arange(N - val_size, N))]

# =========================================================
#  FAST CORE(+exp) 파생피처 (선택 사용)
# =========================================================
EPS_DIV   = 1e-3
CORE_COLS = ["M4", "V7", "E20", "E9", "P5", "E19"]
EXP_SCALE = 1.0
EXP_CLIP  = (-8.0, 8.0)  # exp 안정화

def build_coreexp_pairwise_features_fast(
    df: pd.DataFrame,
    core_cols=CORE_COLS,
    eps: float = EPS_DIV,
    exp_scale: float = EXP_SCALE,
    exp_clip: tuple[float, float] = EXP_CLIP,
    dtype=np.float32
) -> pd.DataFrame:
    present = [c for c in core_cols if c in df.columns]
    if not present:
        return pd.DataFrame(index=df.index)
    vals = {c: pd.to_numeric(df[c], errors="coerce").astype(dtype).to_numpy() for c in present}
    for c in present:
        v = vals[c]
        v_scaled = np.clip(v * exp_scale, exp_clip[0], exp_clip[1])
        vals[f"exp_{c}"] = np.exp(v_scaled).astype(dtype)
    base = list(vals.keys())
    cols = {name: vals[name] for name in base}
    for a, b in combinations(base, 2):
        av, bv = vals[a], vals[b]
        cols[f"{a}_plus_{b}"]   = (av + bv).astype(dtype)
        cols[f"{a}_minus_{b}"]  = (av - bv).astype(dtype)
        cols[f"{b}_minus_{a}"]  = (bv - av).astype(dtype)
        cols[f"{a}_times_{b}"]  = (av * bv).astype(dtype)
        cols[f"{a}_div_{b}"]    = (av / (np.abs(bv) + eps)).astype(dtype)
        cols[f"{b}_div_{a}"]    = (bv / (np.abs(av) + eps)).astype(dtype)
    out = pd.DataFrame(cols, index=df.index).replace([np.inf, -np.inf], np.nan).fillna(0.0)
    return out

X_coreexp = build_coreexp_pairwise_features_fast(train_proc, CORE_COLS, EPS_DIV, EXP_SCALE, EXP_CLIP)
const_cols_coreexp = [c for c in X_coreexp.columns if X_coreexp[c].nunique(dropna=False) <= 1]
if const_cols_coreexp:
    X_coreexp = X_coreexp.drop(columns=const_cols_coreexp)
y_coreexp = y

# Similarity-pruned selection
SIM_THRESH = 0.85
TOPK       = 200
if X_coreexp.shape[1] > 0:
    corr_to_y   = X_coreexp.corrwith(y_coreexp).abs().fillna(0.0)
    corr_ranked = corr_to_y.sort_values(ascending=False)
    corr_mat    = X_coreexp.corr().abs().fillna(0.0)
    selected = []
    for feat in corr_ranked.index:
        if len(selected) >= TOPK:
            break
        if not selected:
            selected.append(feat); continue
        if corr_mat.loc[feat, selected].max() >= SIM_THRESH:
            continue
        selected.append(feat)
    X_coreexp_sel = X_coreexp[selected].copy()
    _feat_cols_coreexp_sel = list(X_coreexp_sel.columns)
    print(f"[CORE+EXP] raw={X_coreexp.shape[1]}, selected={X_coreexp_sel.shape[1]} (sim_thresh={SIM_THRESH}, topk={TOPK})")
else:
    X_coreexp_sel = pd.DataFrame(index=train_proc.index)
    _feat_cols_coreexp_sel = []

# =========================================================
#  A) Base set — train with best params
# =========================================================
_models = []
cv_r2_list = []
for f, (tr_idx, va_idx) in enumerate(folds, 1):
    m = lgb.LGBMRegressor(
        **build_lgbm_params(SEED),
        n_estimators=5000,
        random_state=SEED,
        n_jobs=-1,
        verbosity=-1,
    )
    m.fit(
        X_base.iloc[tr_idx], y.iloc[tr_idx],
        eval_set=[(X_base.iloc[va_idx], y.iloc[va_idx])],
        eval_metric="rmse",
        callbacks=[
            lgb.early_stopping(stopping_rounds=EARLY_STOP, verbose=False),
            lgb.log_evaluation(LOG_EVERY),
        ],
    )
    best_it = getattr(m, "best_iteration_", None)
    y_va_pred = m.predict(X_base.iloc[va_idx], num_iteration=best_it)
    r2 = float(r2_score(y.iloc[va_idx], y_va_pred)) if y.iloc[va_idx].var() > 0 else float("nan")
    print(f"[A-Base][Fold {f}] best_it={best_it}, R2_valid={r2:.6f}")
    cv_r2_list.append(r2)
    _models.append(m)
valsA = [v for v in cv_r2_list if np.isfinite(v)]
print(f"[A-Base][CV] R2 mean={np.mean(valsA):.6f}, median={np.median(valsA):.6f}" if valsA else "[A-Base][CV] all NaN")

# =========================================================
#  B) Core+Exp pruned — train (선택적)
# =========================================================
models_coreexp_sel = []
cv_r2_coreexp_sel = []
if _feat_cols_coreexp_sel:
    for f, (tr_idx, va_idx) in enumerate(folds, 1):
        m = lgb.LGBMRegressor(
            **build_lgbm_params(SEED),
            n_estimators=5000,
            random_state=SEED,
            n_jobs=-1,
            verbosity=-1,
        )
        m.fit(
            X_coreexp_sel.iloc[tr_idx], y_coreexp.iloc[tr_idx],
            eval_set=[(X_coreexp_sel.iloc[va_idx], y_coreexp.iloc[va_idx])],
            eval_metric="rmse",
            callbacks=[
                lgb.early_stopping(stopping_rounds=EARLY_STOP, verbose=False),
                lgb.log_evaluation(LOG_EVERY),
            ],
        )
        best_it = getattr(m, "best_iteration_", None)
        yhat = m.predict(X_coreexp_sel.iloc[va_idx], num_iteration=best_it)
        r2 = float(r2_score(y_coreexp.iloc[va_idx], yhat)) if y_coreexp.iloc[va_idx].var() > 0 else float("nan")
        cv_r2_coreexp_sel.append(r2)
        models_coreexp_sel.append(m)
        print(f"[B-Core+Exp][Fold {f}] best_it={best_it}, R2_valid={r2:.6f}")
    valsB = [v for v in cv_r2_coreexp_sel if np.isfinite(v)]
    print(f"[B-Core+Exp][CV] R2 mean={np.mean(valsB):.6f}, median={np.median(valsB):.6f}" if valsB else "[B-Core+Exp][CV] all NaN")

# =========================================================
#  OoF Calibration (확률 p, 기대수익 μ) + payoff 추정(g_up, g_down)
# =========================================================
_y_oof = np.full(len(X_base), np.nan, dtype=float)
_yhat_oof = np.full(len(X_base), np.nan, dtype=float)

for (tr_idx, va_idx), m in zip(folds, _models):
    best_it = getattr(m, "best_iteration_", None)
    _y_oof[va_idx] = y.iloc[va_idx].to_numpy()
    _yhat_oof[va_idx] = m.predict(X_base.iloc[va_idx], num_iteration=best_it)

_mask = np.isfinite(_y_oof) & np.isfinite(_yhat_oof)
y_oof = _y_oof[_mask]
yhat_oof = _yhat_oof[_mask]

_cal_logit = None
_cal_lin   = None
_g_up = 0.001
_g_down = 0.001

if y_oof.size >= 200:
    lbl = (y_oof > 0).astype(int)
    _cal_logit = LogisticRegression(max_iter=2000)
    _cal_logit.fit(yhat_oof.reshape(-1,1), lbl)

    _cal_lin = LinearRegression()
    _cal_lin.fit(yhat_oof.reshape(-1,1), y_oof)

    # 절단평균으로 양/음 평균 크기 추정
    def _trimmed_mean(v):
        lo, hi = np.nanpercentile(v, [5,95])
        vv = v[(v>=lo)&(v<=hi)]
        return float(np.nanmean(vv)) if vv.size>0 else float(np.nanmean(v))
    pos = y_oof[y_oof>0]
    neg = -y_oof[y_oof<0]
    _g_up   = max(1e-4, _trimmed_mean(pos) if pos.size>0 else 0.001)
    _g_down = max(1e-4, _trimmed_mean(neg) if neg.size>0 else 0.001)
else:
    print("[cal] OoF insufficient -> use conservative defaults.")

def _prob_up_from_yhat(yhat: float) -> float:
    if _cal_logit is None:
        # fallback: 완만한 sigmoid
        return float(1.0/(1.0+np.exp(-yhat)))
    return float(_cal_logit.predict_proba([[yhat]])[0,1])

def _mu_from_yhat(yhat: float) -> float:
    if _cal_lin is None:
        return float(yhat)
    return float(_cal_lin.predict([[yhat]])[0])

def _kelly_multiplier_from_proba(p: float, g_up: float = None, g_down: float = None, cap: float = 2.0) -> float:
    """Kelly for asymmetric payoffs:
       f* = (p*g_up - (1-p)*g_down) / (g_up * g_down), multiplier = 1 + f*
       (long-only → 음수일 땐 0으로 클립)"""
    gu = _g_up if g_up is None else g_up
    gd = _g_down if g_down is None else g_down
    gu = max(gu, 1e-8); gd = max(gd, 1e-8)
    f_star = (p*gu - (1.0-p)*gd) / (gu*gd)
    mult = 1.0 + f_star
    return float(np.clip(mult, 0.0, cap))

# =================================================
# Vol guard helpers
# =================================================
_realized_window = []  # 최근 일일 실현 수익률
def _update_vol_guard(df_row: pd.DataFrame, current_alloc: float):
    global _realized_window
    # lagged_forward_returns 또는 market_forward_excess_returns 사용
    val = None
    for c in ["lagged_forward_returns", "lagged_market_forward_excess_returns"]:
        if c in df_row.columns:
            try:
                val = float(df_row.iloc[0][c])
                break
            except Exception:
                pass
    if val is None or not np.isfinite(val):
        return
    _realized_window.append(val)
    if len(_realized_window) > VOL_GUARD_WIN:
        _realized_window = _realized_window[-VOL_GUARD_WIN:]

def _vol_damper():
    if TARGET_DAILY_VOL <= 0 or len(_realized_window) < max(10, VOL_GUARD_WIN//5):
        return 1.0
    vol = float(np.std(_realized_window, ddof=0))
    if vol <= 1e-8:
        return 1.0
    return float(np.clip(TARGET_DAILY_VOL / vol, 0.0, 1.0))

# =================================================
# Predict (called once per new row)
# =================================================
def predict(test_row):
    global _test_buffer, _last_allocation, _prev_raw_pred_sign

    # (0) to pandas & append to buffer
    test_pd = _to_pandas(test_row)
    test_pd.columns = [str(c) for c in test_pd.columns]

    # (0-a) yesterday realized return vs previous sign (히트율 추적 — 선택)
    realized_prev = None
    if "lagged_forward_returns" in test_pd.columns:
        realized_prev = test_pd.loc[0, "lagged_forward_returns"]
    elif "lagged_market_forward_excess_returns" in test_pd.columns:
        realized_prev = test_pd.loc[0, "lagged_market_forward_excess_returns"]
    if _prev_raw_pred_sign is not None and pd.notna(realized_prev):
        r = float(realized_prev)
        if r != 0.0:
            _kelly_hits.append(1 if np.sign(r) == _prev_raw_pred_sign else 0)

    # update buffer & re-run SAME pipeline
    _test_buffer = pd.concat([_test_buffer, test_pd], axis=0, ignore_index=True)
    buf_proc = _online_pipeline_apply(_test_buffer)
    last_proc = buf_proc.tail(1).copy()
    X_last = _align_features(last_proc)

    # --- A) Base tuned ensemble ---
    preds = []
    for m in _models:
        best_it = getattr(m, "best_iteration_", None)
        preds.append(float(m.predict(X_last, num_iteration=best_it)[0]))
    raw_pred = float(np.mean(preds)) if preds else 0.0

    # --- (선택) B) Core+Exp tuned로 대체하려면 아래 주석 해제 ---
    # if _feat_cols_coreexp_sel:
    #     last_B = build_coreexp_pairwise_features_fast(last_proc, CORE_COLS, EPS_DIV, EXP_SCALE, EXP_CLIP)
    #     for c in _feat_cols_coreexp_sel:
    #         if c not in last_B.columns: last_B[c] = 0.0
    #     last_B = last_B.reindex(columns=_feat_cols_coreexp_sel, fill_value=0.0)
    #     predsB = [float(m.predict(last_B, num_iteration=getattr(m,"best_iteration_",None))[0]) for m in models_coreexp_sel]
    #     raw_pred = float(np.mean(predsB)) if predsB else raw_pred

    _prev_raw_pred_sign = 0 if raw_pred == 0 else (1 if raw_pred > 0 else -1)

    # ====== Calibration으로 P(up), μ 생성 ======
    p_up = _prob_up_from_yhat(raw_pred)   # 일일 상승확률
    mu   = _mu_from_yhat(raw_pred)        # 일일 기대 수익률

    # ====== 기본 스케일 × Kelly(최대 2x) × Vol guard ======
    scale = _choose_scale(last_proc.iloc[0])
    base_alloc = np.clip(mu * scale, CLIP_MIN, CLIP_MAX)     # μ 기반 베이스 포지션
    kelly_mult = _kelly_multiplier_from_proba(p_up, cap=2.0) # 0~2 배
    vol_mult   = _vol_damper()                               # 0~1 배
    alloc = np.clip(base_alloc * kelly_mult * vol_mult, CLIP_MIN, CLIP_MAX)

    # 스무딩
    alloc = (1.0 - SMOOTH_ALPHA) * alloc + SMOOTH_ALPHA * _last_allocation
    _last_allocation = float(alloc)

    # vol guard 업데이트
    _update_vol_guard(test_pd, _last_allocation)

    return float(alloc)

# ============== Launch server ==============
inference_server = kaggle_evaluation.default_inference_server.DefaultInferenceServer(predict)
if os.getenv("KAGGLE_IS_COMPETITION_RERUN"):
    inference_server.serve()
else:
    # 로컬에서 submission.parquet 생성
    import pyarrow  # parquet 엔진 확인 (Kaggle에는 기본 설치됨)

    test = pd.read_csv(TEST_CSV).sort_values(TIME_COL).reset_index(drop=True)

    # 상태 초기화(선택): 온라인 버퍼/가드 리셋
    _test_buffer = pd.DataFrame()
    _realized_window = []
    _last_allocation = 0.0
    _prev_raw_pred_sign = None

    preds = []
    for _, row in test.iterrows():
        # predict는 1-행 dict/Series/DataFrame 모두 지원하도록 위에서 wrapper 구성됨
        preds.append(predict(row.to_dict()))

    sub = pd.DataFrame({"allocation": preds})
    # ID 컬럼 붙이기(대회 스키마에 맞춰 자동 선택)
    if "row_id" in test.columns:
        sub.insert(0, "row_id", test["row_id"].values)
    elif "prediction_id" in test.columns:
        sub.insert(0, "prediction_id", test["prediction_id"].values)
    elif "date_id" in test.columns:
        sub.insert(0, "date_id", test["date_id"].values)

    sub.to_parquet("submission.parquet", index=False)
    print("[Saved] submission.parquet", sub.shape)


In [ ]:
def exp_saturation_report(df_full: pd.DataFrame, folds, tol=0.1):
    # exp_* 피처의 log가 clip 경계(±8)에 얼마나 붙는지 비율
    cols = [c for c in df_full.columns if c.startswith("exp_")]
    if not cols: 
        print("[SAT] no exp_* columns found"); 
        return
    for f, (_, va_idx) in enumerate(folds, 1):
        sub = df_full.iloc[va_idx][cols]
        # log(exp_x) = x_scaled (clip된 값에 근사)
        logv = np.log(np.clip(sub, 1e-12, None))
        lower = (logv <= (EXP_CLIP[0] + tol)).mean().mean()
        upper = (logv >= (EXP_CLIP[1] - tol)).mean().mean()
        print(f"[SAT][Fold {f}] near-lower≈{lower:.3f}, near-upper≈{upper:.3f}, total≈{(lower+upper):.3f}")

# 원본(exp 포함) 전체 파생 DataFrame은 X_coreexp 이고,
# pruned는 그 중 30개 선택 열만이지만 포화율은 전체 exp_*에서 보는 게 좋아요.
exp_cols = [c for c in X_coreexp.columns if c.startswith("exp_")]
if exp_cols:
    exp_saturation_report(X_coreexp[exp_cols], folds)


In [ ]:
# ===== Extra diagnostics for the pruned model =====
import numpy as np, pandas as pd

def prune_fold_diagnostics(X_sel: pd.DataFrame, y: pd.Series, models):
    rows = []
    for f, (tr_idx, va_idx) in enumerate(folds, 1):
        X_va, y_va = X_sel.iloc[va_idx], y.iloc[va_idx]
        m = models[f-1]
        bi = getattr(m, "best_iteration_", None)
        yhat = m.predict(X_va, num_iteration=bi)

        y_std    = float(y_va.std())
        yhat_std = float(np.std(yhat))
        rmse     = float(np.sqrt(np.mean((y_va - yhat)**2)))
        used_cnt = int((m.booster_.feature_importance(importance_type="split") > 0).sum())

        rows.append(dict(fold=f, best_it=bi, y_std=y_std, yhat_std=yhat_std,
                         rmse=rmse, used_feats=used_cnt))
    df = pd.DataFrame(rows)
    print("\n[PRUNE][DIAG]\n", df.to_string(index=False))
    df.to_csv("/kaggle/working/prune_fold_diagnostics.csv", index=False)

prune_fold_diagnostics(X_coreexp_sel, y_coreexp, models_coreexp_sel)
